In [ ]:
# Modelagem de dados
import pandas             as pd
import numpy              as np
from numpy                import *

# Análises graficas
import matplotlib.pyplot  as plt
import seaborn            as sns
import plotly.express     as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.image   as mpimg

# Ajuste no matplot
plt.rcParams['figure.figsize'] = (15,6)
plt.style.use('ggplot')

# Machine Learnig
from scipy.sparse         import csc_matrix
from sklearn.neighbors    import NearestNeighbors
from scipy.optimize       import brute

# Framework
import PIL
import urllib
import requests

## Importação dos dados

In [ ]:
# Ler os dados

dados_livros = pd.read_csv(r'Dados_Recomendacao - Livro\Books.csv', low_memory=False)
dados_avaliacao = pd.read_csv(r'Dados_Recomendacao - Livro\Ratings.csv')
dados_usuarios = pd.read_csv(r'Dados_Recomendacao - Livro\Users.csv')

dados_livros.shape, dados_avaliacao.shape, dados_usuarios.shape

### Verificando os dados

In [ ]:
dados_livros.info()

In [ ]:
dados_livros.head()

In [ ]:
dados_avaliacao.info()

In [ ]:
dados_avaliacao.head()

In [ ]:
dados_usuarios.info()

In [ ]:
dados_usuarios.head()

## Modelagem dos dados

In [ ]:
# Cruzando os dados dos livros com os dados de avaliação
tab_cruzada = dados_livros.merge(dados_avaliacao, how='inner', on='ISBN')

# Cruzando os dados dos livros com os dados dos usuarios
tab_cruzada = tab_cruzada.merge(dados_usuarios, how='inner', on='User-ID')

In [ ]:
tab_cruzada.shape

In [ ]:
tab_cruzada.head()

In [ ]:
tab_cruzada.info()

In [ ]:
## Investigação dos dados ##

# for linha in tab_cruzada['Year-Of-Publication'].value_counts().index:
#   print(linha)

# tab_cruzada.loc[tab_cruzada['Year-Of-Publication'] == 'DK Publishing Inc']

# Ajustando o registro incorreto
tab_cruzada.iloc[287500, 3] = ''
tab_cruzada.iloc[352361, 3] = ''
tab_cruzada.iloc[467962, 3] = ''
tab_cruzada.iloc[469216, 3] = ''

tab_cruzada['Year-Of-Publication'] = pd.to_numeric(tab_cruzada['Year-Of-Publication'])

In [ ]:
tab_cruzada.dtypes

In [ ]:
tab_cruzada['Location'].head()

In [ ]:
tab_cruzada['Location'].tail()

In [ ]:
# Tratamento de texto

def extrair_pais(regiao):
    '''
        Funcao para extrair o nome do pais na coluna regiao
    '''

    # Incluindo a informação
    registro = regiao 

    # Fatiar
    registro = regiao.split(',')

    #Buscar
    fracao = registro[-1].upper()
    
    return fracao

tab_cruzada['Pais'] = tab_cruzada['Location'].apply(extrair_pais)

In [ ]:
tab_cruzada.head()

In [ ]:
# Nulos
tab_cruzada.isnull().sum()

In [ ]:
tab_cruzada.nunique()

## Visualização dos dados

In [ ]:
# Análise descritiva

tab_cruzada.describe()

In [ ]:
tab_cruzada = tab_cruzada.loc[tab_cruzada['Book-Rating'] > 0]

tab_cruzada.isnull().sum(), tab_cruzada.shape

In [ ]:
tab_cruzada['Book-Rating'].describe()

In [ ]:
# Análise gráfica

plt.title('Analisando a avaliação')
sns.boxplot(data=tab_cruzada, x = 'Book-Rating')

In [ ]:
# Analise

analise = tab_cruzada.groupby(by= 'Book-Title').agg(
    quantidade = ('Book-Title', 'count'),
    media = ('Book-Rating', 'mean'),
    max = ('Book-Rating', 'max'),
    min = ('Book-Rating', 'min'),
    mediana = ('Book-Rating', 'median'),
)
analise.head()

In [ ]:
analise.sort_values('quantidade', ascending=False).head()

In [ ]:
analise.sort_values(['mediana', 'quantidade'], ascending=False).head()

In [ ]:
# Analise Qtd vs Avaliação

px.scatter(
    data_frame = analise,
    x = 'quantidade', y = 'media',
    title = 'Média x Quantidade - Titulos',
)

In [ ]:
# Correlação
analise.corr('spearman')

In [ ]:
analise['quantidade'].describe()

In [ ]:

def classificacao_quantidade(quantidade):
    ''''
    Agrupar a quantidade
    '''

    if int(quantidade) <= 5:
        return '1 - 5 Avaliações'
    elif int(quantidade) <= 10:
        return '6 - 10 Avaliações'
    elif int(quantidade) <= 50:
        return '11 - 50 Avaliações'
    elif int(quantidade) <= 100:
        return '51 - 100 Avaliações'
    else:
        return '101 Avaliações'

pizza = analise['quantidade'].apply(classificacao_quantidade).value_counts(normalize=True)

pizza = pd.DataFrame(pizza).reset_index()

px.pie(
    data_frame=pizza,
    names='index', values='quantidade',
    title='Divisão das Quantidades'
)

In [ ]:
# Verificando

pizza.head()

In [ ]:
analise_ano = tab_cruzada['Year-Of-Publication'].value_counts().sort_index().reset_index()

analise_ano.describe()

In [ ]:
# Plot

filtro = analise_ano.loc[(analise_ano['index'] > 1990) & (analise_ano['index'] < 2020 )]

plt.title('Analisando ano da publicação')
plt.bar(filtro['index'], filtro['Year-Of-Publication']);

In [ ]:
tab_cruzada.groupby(by='Book-Author').agg(
    quantidade = ('Book-Rating', 'count'),
    media = ('Book-Rating', 'mean')
).sort_values('quantidade', ascending=False).head(10)

In [ ]:
# Concentração das avaliações

tab_cruzada['Pais'].value_counts(normalize=True).head(10) * 100

In [ ]:
# Concentração das avaliações acumulado

tab_cruzada['Pais'].value_counts(normalize=True).cumsum().head(10) * 100

In [ ]:
# Idade

plt.title('Análise de Idades')
sns.boxplot(data=tab_cruzada, x='Age')

## Construção do Modelo

In [ ]:
# Ajustar a avaliação dos livros --> tab_cruzada

avaliacoes = analise.reset_index().iloc[:,0:2]

df_final = tab_cruzada.merge(avaliacoes, how='inner', on='Book-Title')

df_final.head()

In [ ]:
# Filtrar os livros com menos de 50 avaliações

livros_avaliados = df_final.loc[df_final['quantidade'] >= 50]

livros_avaliados.shape

In [ ]:
# Duplicados

livros_avaliados.duplicated().sum()

In [ ]:
# Gerando a matriz

matriz = livros_avaliados.pivot_table(values='Book-Rating', index='Book-Title', columns='User-ID')

matriz.fillna(0, inplace=True)

matriz.head()

In [ ]:
# Transformação para vetores

matriz_sparse = csc_matrix(matriz)

matriz_sparse

In [ ]:
# Exemplo da função

csc_matrix( (4,4), dtype=np.int8).toarray()

In [ ]:
# Criar o modelo

modelo = NearestNeighbors(
    # Qtd de recomendações
    n_neighbors=5,
    # Algoritmo
    algorithm='brute',
    # Metrica da distancia
    metric='minkowski'
)

modelo.fit(matriz_sparse)

In [ ]:
# Recomendações
# Escolher livro
# 
# # Descobrir o livro do Harry Potter
for posicao, Book_Title in enumerate(matriz.index):

    if 'Harry' in Book_Title:
        print(posicao, Book_Title)

In [ ]:
# Selecionar o livro ########## O CLIENTE COMPROU ###########
selecionar_livro = matriz.iloc[2, :].values.reshape(1, -1)

# Previsão do modelo
distancia, recomendacao = modelo.kneighbors(selecionar_livro)

#### AVALIAÇÃO / SERIES / RENTABILIDADE / NOTICIAS ####

# Ver as sugestões
for loop in range(len(recomendacao)):

    print(matriz.index[recomendacao[loop]])

## Simulador

In [ ]:
# Filtrando o link da imagem da capa do livro

link = tab_cruzada.loc[tab_cruzada['Book-Title'] == '2nd Chance'].head(1)['Image-URL-L'].values[0]

# Buscar as info dessa imagem

imagem = PIL.Image.open(urllib.request.urlopen(link))
imagem

In [ ]:
# Lista para receber os links das recomendações
lista_URL = []

# Loop nas recomendações
for loop in range(len(recomendacao)):

    selecao_loop = (matriz.index[recomendacao[loop]])

    lista_URL = selecao_loop

# Filtrando o link das recomendaões
link_recomendacao_1 = link_selecao = tab_cruzada.loc[tab_cruzada['Book-Title'] == lista_URL[1]].head(1)['Image-URL-L'].values[0]
link_recomendacao_2 = link_selecao = tab_cruzada.loc[tab_cruzada['Book-Title'] == lista_URL[2]].head(1)['Image-URL-L'].values[0]
link_recomendacao_3 = link_selecao = tab_cruzada.loc[tab_cruzada['Book-Title'] == lista_URL[3]].head(1)['Image-URL-L'].values[0]
link_recomendacao_4 = link_selecao = tab_cruzada.loc[tab_cruzada['Book-Title'] == lista_URL[4]].head(1)['Image-URL-L'].values[0]

In [ ]:
# Atribuindo as imagens

imagem_01 = PIL.Image.open(urllib.request.urlopen(link_recomendacao_1))
imagem_02 = PIL.Image.open(urllib.request.urlopen(link_recomendacao_2))
imagem_03 = PIL.Image.open(urllib.request.urlopen(link_recomendacao_3))
imagem_04 = PIL.Image.open(urllib.request.urlopen(link_recomendacao_4))

In [ ]:
from matplotlib.pyplot import title

titulos = ['Seleção', 'Recomendação 1', 'Recomendação 2', 'Recomendação 3','Recomendação 4']

# Criando a figura
figura = make_subplots(
    rows=1,
    cols=5,
    subplot_titles=titulos
)

# Ajustando o layout
figura.update_layout(
    height=500,
    width=1200,
    title_text='Sistema de recomendação',
    showlegend=False
)

# Imagem da seleção
figura.add_trace(
    go.Image(
        z=imagem,
    ),
    row=1, col=1
)

# Imagem da recomendação 1
figura.add_trace(
    go.Image(
        z=imagem_01,
    ),
    row=1, col=2
)

# Imagem da recomendação 2
figura.add_trace(
    go.Image(
        z=imagem_02,
    ),
    row=1, col=3
)

# Imagem da recomendação 3
figura.add_trace(
    go.Image(
        z=imagem_03,
    ),
    row=1, col=4
)

# Imagem da recomendação 4
figura.add_trace(
    go.Image(
        z=imagem_04,
    ),
    row=1, col=5
)

# Mostrar
figura.show()